This code contains the code to create & update closing price data for any index lists in the 'Company lists' folder 

In [1]:
#Import the libraries
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
import math
from datetime import timedelta 

pd.set_option('display.max_columns', 38)

## Defining Database & data fetching functions 

In [3]:
DATE_FORMAT = '%Y-%m-%d'

# Data base maintainance functions

#Connects to a the pre-existing CSV price database
def connectAndLoadDb(exchange):
    print("Connecting database:"+str(exchange))
    filename='Price Databases\database_'+str(exchange)+'.csv'
    database = pd.read_csv(filename,index_col=False) 
    print("Database connected!")
    return database;

#Gets the latest date of data in the db
def getLastEntryDate(database):
    lastDateEntry = database.iloc[-1]['Date']
    lastDateEntry = datetime.datetime.strptime(lastDateEntry, DATE_FORMAT)    
    return lastDateEntry

#Writes the updated pandas dataframe to the CSV
def writeDbToExcelFile(database,exchange):
    filename='Price Databases\database_'+str(exchange)+'.csv'
    print('Writing database to filename: '+ filename)
    database.index=database['Date']
    database.drop(['Date'],axis=1,inplace=True)
    database.to_csv(filename)
    print('Database updated with new entries!!')

#Formats the date from number for printing      
def prettyPrintDate(date):
    return date.strftime(DATE_FORMAT);

#Data Fetching functions

#get ticker list from our tsv files
def getTickers(exchange):
    #We have the lists saved as TSV ie delimited wth tabs rather than commas
    df_info=pd.read_csv('Company lists/companylist_'+str(exchange)+'.tsv',sep='\t') 
    l_tickers=df_info.Symbol.tolist()
    return l_tickers;

#Pulls adj closing price data from yfinance for a given list of stock tickers 'l_tickers', for all dates up to today from a given 'date' merges this data with an existing database 'database'
def fetchAndAppendToDb(date, database, exchange):  
    dateStr = prettyPrintDate(date)
    print('Fetching stock closing price of '+str(exchange)+' for days over: ' + dateStr)

    l_tickers=getTickers(exchange)
    #Pulling adj closing price data from yfinance
    mergedData = yf.download(l_tickers,date)['Adj Close']

    #Making date the index col
    mergedData['Date']=mergedData.index

    #append our new data onto the existing databae
    database = database.append(mergedData, ignore_index=True)
    return database;


def fetchData(database,exchange, refetchAll = False):
    if refetchAll == True:
        lastEntryDate = datetime.datetime.strptime('2006-01-01', DATE_FORMAT) #Start date here 
    else:
        lastEntryDate = getLastEntryDate(database)
    ydaysDate = datetime.datetime.today() - timedelta(days = 1)
    if lastEntryDate >= ydaysDate:
        print('Data already loaded up to Yesterday')
        return database
    else:
        print("Last entry in Db is of :" + prettyPrintDate(lastEntryDate))
        print("----------------------------------------------")
    
        dateToFetch = lastEntryDate + timedelta(days=1)

        database = fetchAndAppendToDb(dateToFetch, database, exchange)
    
        print("----------------------------------------------")
        print("Data fill completed! 👍👍")
        return database;

## Update Nasdaq data

In [36]:
%%time
#run this lines for a fresh run
database = pd.DataFrame()
database = fetchData(database,'NASDAQ', refetchAll = True)


# run these lines to update an existing database
#database = connectAndLoadDb('NASDAQ')
#database = fetchData(database,'NASDAQ')

database.tail()

Last entry in Db is of :2006-01-01
----------------------------------------------
Fetching stock closing price of NASDAQ for days over: 2006-01-02
[*********************100%***********************]  4227 of 4227 completed
----------------------------------------------
Data fill completed! 👍👍
Wall time: 15min 39s


,AACG,AACQ,AACQU,AACQW,AAL,AAME,AAOI,AAON,AAPL,AAWW,AAXJ,ABCB,ABCL,ABCM,ABEO,ABGI,ABIO,ABMD,ABNB,...,ZIONP,ZIOP,ZIXI,ZKIN,ZLAB,ZM,ZNGA,ZNTE,ZNTEU,ZNTEW,ZNTL,ZS,ZSAN,ZUMZ,ZVO,ZWRKU,ZYNE,ZYXI,Date
3818,5.15,11.23,12.01,NaN,21.180000,4.76,9.020,77.379997,127.790001,55.320000,97.339996,49.020000,39.689999,22.590000,2.34,10.25,4.37,324.260010,196.419998,...,24.832500,5.26,7.42,7.44,162.020004,409.660004,11.56,10.96,12.14,2.71,41.730000,210.160004,1.33,47.060001,5.19,10.330,4.66,14.69,2021-03-01
3819,5.07,10.73,11.51,NaN,21.440001,4.70,8.625,74.919998,125.120003,56.570000,95.930000,48.970001,38.000000,22.920000,2.36,10.14,4.52,325.309998,189.899994,...,24.910000,5.07,7.35,6.40,148.009995,372.790009,11.39,10.57,11.55,2.40,43.500000,202.610001,1.25,46.540001,5.29,10.150,4.68,14.25,2021-03-02
3820,4.63,10.38,11.03,NaN,22.170000,4.43,8.360,74.089996,122.059998,57.189999,95.620003,50.160000,35.669998,22.209999,2.19,10.09,4.27,307.600006,180.399994,...,24.934999,4.68,6.97,6.29,140.669998,341.570007,11.09,10.35,11.06,1.83,43.630001,187.830002,1.22,46.330002,5.09,10.025,4.51,14.25,2021-03-03
3821,4.26,10.29,10.79,NaN,21.240000,3.65,7.990,72.070000,120.129997,55.139999,93.389999,49.860001,30.049999,21.129999,2.08,9.99,3.98,295.630005,180.229996,...,25.110001,4.31,7.13,5.20,133.070007,343.089996,10.48,10.20,10.95,1.51,43.549999,180.899994,1.07,45.389999,4.61,10.000,4.09,14.54,2021-03-04
3822,4.08,10.66,11.34,1.91,20.450001,3.67,8.045,72.800003,121.419998,57.650002,94.459999,50.349998,28.610001,21.110001,2.01,9.97,4.12,301.220001,179.809998,...,24.760000,4.29,7.15,5.18,137.250000,337.429993,10.31,10.36,11.19,1.85,42.330002,175.369995,1.07,46.029999,4.72,10.200,4.15,14.75,2021-03-05


In [37]:
%%time
# Drop the last entry prior to saving as it probably is not a full days data
database.drop(database.tail(1).index, inplace = True) 

# Write the data to CSV
writeDbToExcelFile(database,'Nasdaq')

Writing database to filename: Price Databases\database_Nasdaq.csv
Database updated with new entries!!
Wall time: 24 s


## Update NYSE data

In [38]:
%%time
#run this lines for a fresh run
database = pd.DataFrame()
database = fetchData(database,'NYSE', refetchAll = True)


# run these lines to update an existing database
#database = connectAndLoadDb('NYSE')
#database = fetchData(database,'NYSE')

database.tail()

 SWB.U: No data found, symbol may be delisted
- BSN.U: No data found, symbol may be delisted
- FVT.W: No data found, symbol may be delisted
- NS-A: No data found, symbol may be delisted
- CND.U: No data found, symbol may be delisted
- PEI-C: No data found, symbol may be delisted
- HZA.U: No data found, symbol may be delisted
- PSB-X: No data found, symbol may be delisted
- OAC.Z: No data found, symbol may be delisted
- PSA-H: No data found, symbol may be delisted
- ALU.U: No data found, symbol may be delisted
- AHL-D: No data found, symbol may be delisted
- CIT-B: No data found, symbol may be delisted
- CAS.U: No data found, symbol may be delisted
- FST.U: No data found, symbol may be delisted
- CIM-C: No data found, symbol may be delisted
- PSA-G: No data found, symbol may be delisted
- CHA.U: No data found, symbol may be delisted
- EPR-G: No data found, symbol may be delisted
- HIG-G: No data found, symbol may be delisted
- TNP-F: No data found, symbol may be delisted
- SCV.U: No dat

,A,AA,AAC.U,AAI-B,AAI-C,AAIC,AAN,AAP,AAT,AB,ABB,ABBV,ABC,ABEV,ABG,ABM,ABR,ABR-A,ABR-B,...,YELP,YETI,YEXT,YPF,YRD,YSG,YUM,YUMC,ZBH,ZEN,ZEPP,ZIM,ZNH,ZTO,ZTR,ZTS,ZUO,ZYME,Date
3825,124.480003,25.99,NaN,NaN,NaN,4.06,22.219999,162.660004,31.980000,37.299999,29.280001,108.410004,103.169998,2.50,172.149994,44.029999,16.540001,NaN,NaN,...,37.970001,71.169998,18.010000,4.16,5.22,18.690001,104.510002,59.720001,163.399994,150.539993,17.330000,20.209999,34.560001,35.220001,8.99,157.690002,15.50,36.770000,2021-03-01
3826,123.639999,27.50,NaN,NaN,NaN,4.08,21.760000,164.839996,32.410000,37.529999,29.770000,107.849998,104.250000,2.52,167.699997,43.330002,16.680000,NaN,NaN,...,37.139999,69.760002,17.719999,3.96,4.97,19.290001,104.739998,59.310001,161.190002,148.770004,16.620001,20.459999,33.689999,35.060001,9.02,157.050003,15.29,36.610001,2021-03-02
3827,119.300003,30.83,NaN,NaN,NaN,4.01,22.129999,165.830002,33.900002,37.790001,29.420000,106.709999,103.339996,2.55,166.940002,44.500000,16.370001,NaN,NaN,...,37.529999,69.339996,16.549999,3.99,4.85,18.400000,103.510002,59.610001,159.750000,142.500000,15.510000,21.000000,35.380001,34.799999,9.12,149.320007,15.00,34.160000,2021-03-03
3828,114.989998,28.59,NaN,NaN,NaN,4.01,23.100000,165.539993,33.820000,36.459999,29.170000,105.650002,102.989998,2.66,167.919998,44.439999,15.920000,NaN,NaN,...,37.340000,65.089996,14.570000,4.19,4.46,17.760000,103.570000,59.180000,156.250000,136.130005,14.250000,20.230000,35.299999,34.480000,8.99,144.000000,14.35,31.940001,2021-03-04
3829,116.139999,28.99,NaN,NaN,NaN,4.14,23.959999,171.509995,34.599998,37.389999,29.700001,106.699997,104.830002,2.71,171.649994,46.220001,15.680000,NaN,NaN,...,39.279999,65.629997,15.220000,4.25,4.39,17.799999,104.330002,60.419998,158.500000,133.080002,14.320000,19.299999,33.930000,34.959999,9.14,145.679993,13.65,32.939999,2021-03-05


In [39]:
%%time
# Drop the last entry prior to saving as it probably is not a full days data
database.drop(database.tail(1).index, inplace = True) 

writeDbToExcelFile(database,'NYSE')

Writing database to filename: Price Databases\database_NYSE.csv
Database updated with new entries!!
Wall time: 34.8 s


## Update Other index's data
- A list of the index's Symbols 'Symbol' & Company names 'Description' must first be stored as a TSV at *EF_Portfolio_Optimisation\Company lists* called *companylist_**INDEXNAME(in all caps)**.TSV*

In [ ]:
%%time
indexname='INDEXNAME(in all caps)'
#run this lines for a fresh run
database = pd.DataFrame()
database = fetchData(database,indexname, refetchAll = True)


# run these lines to update an existing database
#database = connectAndLoadDb(indexname)
#database = fetchData(database,indexname)

database.tail()

In [ ]:
%%time
# Drop the last entry prior to saving as it probably is not a full days data
database.drop(database.tail(1).index, inplace = True) 

writeDbToExcelFile(database,indexname)